In [23]:
from pystac_client import Client  
from satsearch import Search
import pandas as pd
from shapely.geometry import Polygon
import requests
import json


### Using the STAC API, find GES DISC Collections

In [3]:
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'
provider_cat = Client.open(STAC_URL)
providers = [p for p in provider_cat.get_children()]

for count, provider in enumerate(providers):
    print(f'{count} - {provider.title}')

0 - LARC_ASDC
1 - USGS_EROS
2 - ESA
3 - GHRC
4 - LAADS
5 - OBPG
6 - OB_DAAC
7 - ECHO
8 - ISRO
9 - LPCUMULUS
10 - EDF_DEV04
11 - GES_DISC
12 - ASF
13 - OMINRT
14 - EUMETSAT
15 - NCCS
16 - NSIDCV0
17 - PODAAC
18 - LARC
19 - USGS
20 - SCIOPS
21 - LANCEMODIS
22 - CDDIS
23 - JAXA
24 - AU_AADC
25 - ECHO10_OPS
26 - LPDAAC_ECS
27 - NSIDC_ECS
28 - ORNL_DAAC
29 - LM_FIRMS
30 - SEDAC
31 - LANCEAMSR2
32 - NOAA_NCEI
33 - USGS_LTA
34 - GESDISCCLD
35 - GHRSSTCWIC
36 - LARC_CLOUD
37 - ASIPS
38 - ESDIS
39 - POCLOUD
40 - NSIDC_CPRD
41 - ORNL_CLOUD
42 - FEDEO
43 - MLHUB
44 - XYZ_PROV
45 - GHRC_DAAC
46 - CSDA
47 - NRSCC
48 - CEOS_EXTRA
49 - AMD_KOPRI
50 - AMD_USAPDC
51 - MOPITT
52 - GHRC_CLOUD
53 - LPCLOUD
54 - CCMEO


In [4]:
gesdisc_stac = Client.open(f'{STAC_URL}/GES_DISC')


In [5]:
collections = [c for c in gesdisc_stac.get_children()]
for c in collections: 
    print(f"{c.id}: {c.title}")


ACOS_L2_Lite_FP.v7.3: ACOS GOSAT/TANSO-FTS Level 2 bias-corrected XCO2 and other select fields from the full-physics retrieval aggregated as daily files V7.3 (ACOS_L2_Lite_FP) at GES DISC
ACOS_L2_Lite_FP.v9r: ACOS GOSAT/TANSO-FTS Level 2 bias-corrected XCO2 and other select fields from the full-physics retrieval aggregated as daily files V9r (ACOS_L2_Lite_FP) at GES DISC
ACOS_L2S.v7.3: ACOS GOSAT/TANSO-FTS Level 2 Full Physics Standard Product V7.3 (ACOS_L2S) at GES DISC
ACOS_L2S.v9r: ACOS GOSAT/TANSO-FTS Level 2 Full Physics Standard Product V9r (ACOS_L2S) at GES DISC
AIRSM_CPR_MAT.v3.2: AIRS-AMSU variables-CloudSat cloud mask, radar reflectivities, and cloud classification matchups V3.2 (AIRSM_CPR_MAT) at GES DISC
AIRS_CPR_IND.v4.0: AIRS-CloudSat cloud mask and radar reflectivities collocation indexes V4.0 (AIRS_CPR_IND) at GES_DISC
AIRS_CPR_MAT.v3.2: AIRS-CloudSat cloud mask, radar reflectivities, and cloud classification matchups V3.2 (AIRS_CPR_MAT) at GES DISC
AIRXAMAP.v005: AIRS/

In [6]:
example_area = gpd.read_file('yosemite.geojson')
with open('yosemite.geojson') as f:
    aoi = json.load(f)['features'][0]
poly = Polygon(aoi['geometry']['coordinates'][0])

todays_date = datetime.today().strftime('%Y-%m-%dT%H:%M:%SZ')


search_params = {
    'bbox': f'{",".join([str(round(f,3)) for f in list(poly.bounds)])}',
    'limit': 5,
    'datetime': f'2014-04-08T00:00:00Z/2015-04-08T00:00:00Z',
    'collections': ['AIRS_CPR_IND.v4.0'],
    'page': 1
}

print(json.dumps(search_params, indent=2))




{
  "bbox": "-119.63,37.813,-119.63,37.813",
  "limit": 5,
  "datetime": "2014-04-08T00:00:00Z/2015-04-08T00:00:00Z",
  "collections": [
    "AIRS_CPR_IND.v4.0"
  ],
  "page": 1
}


In [7]:
search_response = requests.post(f'{STAC_URL}/GES_DISC/search?', json=search_params).json()
print('%d features found with first query' % len(search_response['features']))
feats = search_response['features']


5 features found with first query


In [16]:
feats[0]['links'][0]['href']


for feat in feats:
    print('{:s} {:s} cmr url:  {:s}'.format(
        feat['collection'], feat['properties']['datetime'], feat['links'][0]['href']
    ))


AIRS_CPR_IND.v4.0 2014-04-08T10:41:34.000Z download url:  https://cmr.earthdata.nasa.gov/stac/GES_DISC/collections/AIRS_CPR_IND.v4.0/items/AIRS_CPR_IND.4.0:index-airs.aqua_cloudsat-v4.0-2014.04.08.107.nc4
AIRS_CPR_IND.v4.0 2014-04-08T20:11:34.000Z download url:  https://cmr.earthdata.nasa.gov/stac/GES_DISC/collections/AIRS_CPR_IND.v4.0/items/AIRS_CPR_IND.4.0:index-airs.aqua_cloudsat-v4.0-2014.04.08.202.nc4
AIRS_CPR_IND.v4.0 2014-04-08T21:47:34.000Z download url:  https://cmr.earthdata.nasa.gov/stac/GES_DISC/collections/AIRS_CPR_IND.v4.0/items/AIRS_CPR_IND.4.0:index-airs.aqua_cloudsat-v4.0-2014.04.08.218.nc4
AIRS_CPR_IND.v4.0 2014-04-10T10:29:34.000Z download url:  https://cmr.earthdata.nasa.gov/stac/GES_DISC/collections/AIRS_CPR_IND.v4.0/items/AIRS_CPR_IND.4.0:index-airs.aqua_cloudsat-v4.0-2014.04.10.105.nc4
AIRS_CPR_IND.v4.0 2014-04-10T19:59:34.000Z download url:  https://cmr.earthdata.nasa.gov/stac/GES_DISC/collections/AIRS_CPR_IND.v4.0/items/AIRS_CPR_IND.4.0:index-airs.aqua_cloudsat